<a href="https://colab.research.google.com/github/acts701/Audio/blob/main/250118_FAISS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 7.2 MB/s eta 0:00:00


In [3]:
import os
import json
from sentence_transformers import SentenceTransformer
from google.colab import drive
from tqdm import tqdm

# Google Drive 마운트
drive.mount('/content/drive')

# 사용자 설정
originTxt_folder_path = '/content/drive/MyDrive/LLM/Data/voiceToText/clovaPaid'  # STT 파일 경로
output_folder = '/content/drive/MyDrive/LLM/Data/vectorDBChunks'  # JSON 청크 저장 경로
os.makedirs(output_folder, exist_ok=True)  # 출력 폴더 생성

Mounted at /content/drive


In [5]:
from pinecone import Pinecone, ServerlessSpec
from google.colab import userdata

# Pinecone 설정
key = userdata.get('PINECONE_KEY')  # API Key 가져오기
pc = Pinecone(api_key=key)  # Pinecone 초기화
environment = "us-east-1"  # Pinecone 환경 설정
index_name = userdata.get('PINECONE-KR-INDEX-NAME')  # 인덱스 이름 가져오기

# ServerlessSpec 정의
serverless_spec = ServerlessSpec(
    cloud="aws",       # 클라우드 제공자 (예: 'gcp', 'aws')
    region="us-east-1"  # Pinecone 콘솔에서 확인한 리전
)

# 목표 차원 설정
target_dimension = 768

In [6]:
# 기존 인덱스 확인 및 차원 가져오기
if index_name in pc.list_indexes().names():
    print(f"기존 인덱스 '{index_name}'가 존재합니다. 가져옵니다...")

    # 인덱스 설명 가져오기
    index_stats = pc.describe_index(index_name)
    existing_dimension = index_stats["dimension"]
    print(f"인덱스 '{index_name}'의 차원: {existing_dimension}")

    # 차원 검증
    if existing_dimension != target_dimension:
        raise ValueError(
            f"인덱스 차원({existing_dimension})이 설정된 차원({target_dimension})과 다릅니다. "
            f"새로 생성하거나 올바른 인덱스를 사용하세요."
        )
else:
    print(f"'{index_name}' 인덱스가 존재하지 않습니다. 새로 생성합니다...")
    pc.create_index(
        name=index_name,
        dimension=target_dimension,  # SentenceTransformer 벡터 크기
        metric="cosine",  # 벡터 유사도 계산 방식 (예: cosine)
        spec=serverless_spec  # 클라우드 설정
    )

# 인덱스 가져오기
index = pc.Index(index_name)
print(f"인덱스 '{index_name}'를 사용할 준비가 되었습니다.")

기존 인덱스 'korean768'가 존재합니다. 가져옵니다...
인덱스 'korean768'의 차원: 768
인덱스 'korean768'를 사용할 준비가 되었습니다.


In [7]:
!pip install sentence-transformers

In [8]:
import torch
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
# KoBERT 모델 이름: "skt/kobert-base-v1"
# model_name = "skt/kobert-base-v1"

# 1) 토크나이저와 모델 불러오기
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModel.from_pretrained(model_name)
model = SentenceTransformer("jhgan/ko-sbert-multitask")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.78k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
!pip install openai==0.28.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.59.6
    Uninstalling openai-1.59.6:
      Successfully uninstalled openai-1.59.6


In [10]:
import openai
from google.colab import userdata

# OpenAI API 키 가져오기
api_key = userdata.get('OPENAI_API_KEY')  # Colab에서 저장한 API 키
if api_key is None:
    raise ValueError("API 키를 찾을 수 없습니다. userdata에 API 키를 저장했는지 확인하세요.")

# OpenAI API 키 설정
openai.api_key = api_key

In [36]:
# 검색 테스트
query_text = "이사야의 중요한 주제"
try:
    # 검색 쿼리를 영어로 번역
    # translated_query = translator.translate(query_text, src='ko', dest='en').text
    prompt = f"""
                You're a professional translator. Translate the following Korean sentence into English.
                You must respond with English only, and nothing else.

                Korean text: {query_text}

                Please provide only the English translation, with no extra commentary.
            """
    # ChatGPT 요청
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0125",  # gpt-4 사용 시 모델 변경 가능
        messages=[
            {"role": "system", "content": "You are an expert translator."},
            {"role": "user", "content": prompt}
        ]
    )
    translated_query = response['choices'][0]['message']['content']
    # query_vector = model.encode(translated_query).tolist()
    query_vector = model.encode("The important themes of Isaiah").tolist()
    top_k = 3
    search_results = index.query(
        vector=query_vector,
        top_k=top_k,
        namespace="ns1",
        include_metadata=True
    )

    print("검색 결과:")
    for match in search_results["matches"]:
        print(f"ID: {match['id']}, Score: {match['score']}, Original Text: {match['metadata']['text']}, Translated Text: {match['metadata']['translated_text']}")
except Exception as e:
    print(f"검색 오류: {e}")

검색 결과:
ID: SA_1_1_9.txt-2, Score: 0.681410372, Original Text: 이사야 1장 1절입니다, Translated Text: Isaiah 1:1
ID: SA_2_1_5.txt-1, Score: 0.61779362, Original Text: 아모스의 아들 이사야가 받은 바 유다와 예루살렘에 관한 말씀이라 말리에 여호와의 전의 산이 모든 산 꼭대기에 굳게 설 것이오., Translated Text: The word that Isaiah son of Amoz saw concerning Judah and Jerusalem. In the last days the mountain of the Lord's temple will be established as the highest of the mountains.
ID: SA_1_21_31.txt-5, Score: 0.573399603, Original Text: 그러므로 주 만군의 여호와 이스라엘의 전능자가 말씀하시되 슬프다., Translated Text: Therefore, the Lord, the God of Hosts, the Mighty One of Israel, declares, "Woe to you."


In [14]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 77.2 MB/s eta 0:00:00


In [18]:
import json
import faiss
import numpy as np

# 벡터 데이터와 텍스트 데이터 로드
vector_data = []  # 벡터 리스트
metadata = []  # 메타데이터 리스트

for file_name in os.listdir(output_folder):
    if file_name.endswith('.json'):
        file_path = os.path.join(output_folder, file_name)
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            for item in data:
                vector_data.append(item['vector'])
                metadata.append({"id": item['id'], "text": item['text'], "translated_text": item.get('translated_text', '')})

# FAISS 인덱스 생성
dimension = len(vector_data[0])
faiss_index = faiss.IndexFlatL2(dimension)  # L2 거리 기반 인덱스
faiss_index.add(np.array(vector_data, dtype='float32'))  # 벡터 추가

In [33]:
# 검색 테스트
query_text = "이사야의 중요한 주제"

try:
    # 검색 쿼리를 영어로 번역
    prompt = f"""
                You're a professional translator. Translate the following Korean sentence into English.
                You must respond with English only, and nothing else.

                Korean text: {query_text}

                Please provide only the English translation, with no extra commentary.
            """
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {"role": "system", "content": "You are an expert translator."},
            {"role": "user", "content": prompt}
        ]
    )
    translated_query = response['choices'][0]['message']['content']
    query_vector = np.array([model.encode(translated_query)], dtype='float32')
    # query_vector = np.array([model.encode(query_text)], dtype='float32')

    # FAISS 검색 수행
    top_k = 3
    distances, indices = faiss_index.search(query_vector, k=top_k)

    print("검색 결과:")
    for i, idx in enumerate(indices[0]):
        result = metadata[idx]
        print(f"Rank {i+1}:\nID: {result['id']}\nOriginal Text: {result['text']}\nTranslated Text: {result['translated_text']}\nDistance: {distances[0][i]}\n")

except Exception as e:
    print(f"검색 오류: {e}")

검색 결과:
Rank 1:
ID: SA_1_21_31.txt-100
Original Text: 이러지 이거는 이건 우리가 이사야에서 이사야님이시니까 이렇게 들어주고 있지.
Translated Text: We should listen to what Isaiah is saying because he is Isaiah.
Distance: 164.3745880126953

Rank 2:
ID: SA_1_21_31.txt-38
Original Text: 이사야 후반부의 주제가 벌써 시작되고 있는 것입니다.
Translated Text: The theme of the latter part of Isaiah has already begun.
Distance: 169.1984100341797

Rank 3:
ID: SA_2_1_5.txt-57
Original Text: 이사야의 중요한 주제가 심판과 구원입니다.
Translated Text: The important themes of Isaiah are judgment and salvation.
Distance: 173.57980346679688

